<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/spacy_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
ls

images/  pending_images/  reviews_text.csv


In [0]:
import pandas as pd
import numpy as np
import random
import json
from pandas.io.json import json_normalize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [74]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
from spacy import displacy



✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
nlp = spacy.load('en_core_web_lg')

In [9]:
df_review = pd.read_csv('reviews_text.csv')
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 36 columns):
Unnamed: 0           56156 non-null int64
_id                  56156 non-null object
updatedAt            56156 non-null object
createdAt            56156 non-null object
originality          56156 non-null int64
value                56156 non-null int64
nutrition            56156 non-null int64
presentation         56156 non-null int64
taste                56156 non-null int64
text                 56155 non-null object
dish                 56156 non-null object
type                 56156 non-null object
isPublished          5885 non-null object
user                 56156 non-null object
restaurant           56156 non-null object
images               56156 non-null object
likes                56156 non-null object
__v                  56156 non-null int64
comments             56156 non-null object
likesCount           56156 non-null int64
tags                 56156 non-null obje

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,25,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
review_text = df_review[['_id','text']]

In [13]:
review_text

,_id,text
0,59e599911747c90004ba8586,Burger joint offers a wide range of cheeseburg...
1,59ff122b8b04fd0004df6c78,It was really good. The mushroom broth was esp...
2,5dc251e552e7e90020b6adfb,"$8.90 for sesame rice, mushroom rendang, curry..."
3,5d35cf4049745d0004c68de3,2 mains + 1 green rice bento from greendot. Re...
4,5d304e9264f5e5000423b75b,The lion mane mushroom rendang is so delicious...
...,...,...
56151,5de6f2e989849e0020715f6b,Good love office treats! These are perfectly s...
56152,5de6f4cf89849e0020716021,Very tasty! Crisp fresh vegetables with fried ...
56153,5de6f53d89849e002071605c,Delicious! Creamy coconut milk red curry sauce...
56154,5de6f69989849e00207160ca,Absolutely delicious! Great if you’re on the r...


In [25]:

review_sample

'This was really good. The hot sauce that it comes with is super hot and I definitely overdid it on the first taco so i was burning up a bit but the meal was great. Can’t say I loved the tortillas so I just scraped off the topping and ate that instead.'

In [73]:
# Select a random review from the corpus
check = random.randint(0,20000)
review_sample = review_text['text'][check]
print(review_sample)

# convert that into nlp
review_doc = nlp(review_sample)

# Break those into sentences
print('----')
for sentence in review_doc.sents:
  print(sentence.text)


# Find the part of speech
print('----')

tokens_list = []
for token in review_doc:
  token_dict = {}
  token_dict['text'] = token.text
  token_dict['lemma'] = token.lemma_
  token_dict['part_of_speech'] = token.pos_
  token_dict['dependency'] = token.dep_
  token_dict['shape'] = token.shape_
  token_dict['alphabet?'] = token.is_alpha
  token_dict['stop_word'] = token.is_stop
  token_dict['punctuation'] = token.is_punct
  token_dict['digit'] = token.is_digit
  token_dict['language'] = token.lang_
  token_dict['prefix'] = token.prefix_
  token_dict['sentiment'] = token.sentiment
  token_dict['currency'] = token.is_currency

  tokens_list.append(token_dict)
df_token = pd.DataFrame(tokens_list)
df_token
# Find the entities in the sample
print('----')
ents = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in review_doc.ents]
print(ents)

This is great! Black beans soy chorizo, peppers, onions, corn, tortilla chips YUM! #jselbys #sprinfarmsanctuary
----
This is great!
Black beans soy chorizo, peppers, onions, corn, tortilla chips YUM!
#jselbys #sprinfarmsanctuary
----


,text,lemma,part_of_speech,dependency,shape,alphabet?,stop_word,punctuation,digit,language,prefix,sentiment,currency
0,This,this,DET,nsubj,Xxxx,True,True,False,False,en,T,0.0,False
1,is,be,VERB,ROOT,xx,True,True,False,False,en,i,0.0,False
2,great,great,ADJ,acomp,xxxx,True,False,False,False,en,g,0.0,False
3,!,!,PUNCT,punct,!,False,False,True,False,en,!,0.0,False
4,Black,black,ADJ,amod,Xxxxx,True,False,False,False,en,B,0.0,False
5,beans,bean,NOUN,compound,xxxx,True,False,False,False,en,b,0.0,False
6,soy,soy,NOUN,compound,xxx,True,False,False,False,en,s,0.0,False
7,chorizo,chorizo,NOUN,ROOT,xxxx,True,False,False,False,en,c,0.0,False
8,",",",",PUNCT,punct,",",False,False,True,False,en,",",0.0,False
9,peppers,pepper,NOUN,conj,xxxx,True,False,False,False,en,p,0.0,False


----
[('YUM', 78, 81, 'ORG'), ('#', 83, 84, 'CARDINAL'), ('#', 92, 93, 'CARDINAL')]


In [0]:
# Import the English language class
from spacy.lang.en import English



In [72]:
similarity_list = []
for token1 in review_doc:
  for token2 in review_doc:
    sim_dict = {}
    sim_dict['word_1'] = token1.text
    sim_dict['word_2'] = token2.text
    sim_dict['similarity'] = token1.similarity(token2)
    similarity_list.append(sim_dict)

df_similarity = pd.DataFrame(similarity_list)\
.query('similarity < 0.9999')\
.sort_values(['similarity'],ascending=False)
df_similarity

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty v

,word_1,word_2,similarity
366,was,had,0.810536
159,had,was,0.810536
246,was,had,0.810536
154,had,was,0.810536
529,wontons,wonton,0.794419
...,...,...,...
22,Best,wontons,-0.083869
549,wontons,of,-0.087127
526,of,wontons,-0.087127
430,at,wontons,-0.092918
